In [1]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhijitdeshpande/Library/Application Support/sagemaker/config.yaml


In [2]:
ROLE='arn:aws:iam::720332985926:role/service-role/AmazonSageMaker-ExecutionRole-20240705T185967'
SOURCE_DIR='/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/src'

In [3]:
sagemaker_session = sagemaker.Session()

In [4]:
model_uri = 's3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-19-23-24-05-737/output/model.tar.gz'

In [5]:
huggingface_model = HuggingFaceModel(
    entry_point='inference.py',
    model_data=model_uri,
    source_dir=SOURCE_DIR,
    role=ROLE,
    transformers_version='4.49.0',
    py_version='py312',
    pytorch_version='2.6.0',
    sagemaker_session=sagemaker_session
    )

In [6]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge'
    )


----------!

In [11]:
predictor.predict({"inputs": "Classify the intent:  Can I cacnel my ticket?"})


{'generated_text': 'a.'}